In [1]:
###let's try to predeict to two seasons ... 

##import files 

In [3]:

import pandas as pd
import numpy as np

In [4]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


In [5]:
##note KNN or other clusters might be helpful group the teams in smart way ... but not now.
#models

##regression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

#classifiers (non-tree)
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor, SGDClassifier
from sklearn.svm import SVC


#tree-based classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import XGBRegressor

##regression models
lr = Ridge(alpha=0.001) 
rfr = RandomForestRegressor(max_depth=3, random_state=0)
xgbr = XGBRegressor()

##classifier models
lrc = RidgeClassifier()
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0)
svc = SVC()

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=3, random_state=0)
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
xgbc = XGBClassifier()



In [6]:

import pandas as pd
import numpy as np

In [7]:
## cleaning tool


def perc_null(X):
    
    total = X.isnull().sum().sort_values(ascending=False)
    data_types = X.dtypes
    percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)

    missing_data = pd.concat([total, data_types, percent], axis=1, keys=['Total','Type' ,'Percent'])
    return missing_data


##TUNING INFO 


##hyper_parameters from here 
##https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
##for xgboost from here 
##https://machinelearningmastery.com/extreme-gradient-boosting-ensemble-in-python/

#xgb

trees = [10, 50, 100, 500, 1000, 5000]  #100  #num of trees
max_depth = range(1,11)  ##3-5
rates = [0.0001, 0.001, 0.01, 0.1, 1.0]  #0.1
subsample in arange(0.1, 1.1, 0.1):  #0.4, 0.5  ##this is 0.1, 0.2 ... 1.0 # % of features to sample


#svc 
kernels in [‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’] #if you use poly, then adjust degree
C in [100, 10, 1.0, 0.1, 0.001]

#gb

learning_rate in [0.001, 0.01, 0.1]
n_estimators [10, 100, 1000]
subsample in [0.5, 0.7, 1.0]
max_depth in [3, 7, 9]


#rfc
max_features [1 to 20]  #key
max_features in [‘sqrt’, ‘log2’]
n_estimators in [10, 100, 1000]

#bc
n_estimators in [10, 100, 1000]

svm_dic = {'kernels':[‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’]}
lrc_dic = {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}
lgr_hp_dic = {'solver': [‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’], 'penalty' : [‘none’, ‘l1’, ‘l2’, ‘elasticnet’],
'C' :[100, 10, 1.0, 0.1, 0.01]}

In [212]:
X = pd.read_csv('/Users/joejohns/data_bootcamp/GitHub/final_project_nhl_prediction/Data/Shaped_Data/data_LJ.csv')
X.shape

(10385, 26)

In [213]:
X['goal_diff_target'] = X['home_goals'] - X['away_goals']


In [214]:
#we will use this for correlations etc at the bottom

data = X.copy()
X.columns

Index(['Unnamed: 0', 'game_id', 'mp_date', 'season', 'home_team', 'away_team',
       'home_odds', 'away_odds', 'home_goals', 'away_goals', 'home_win',
       'settled_in', 'CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv',
       'goal_diff_target'],
      dtype='object')

In [215]:
 
X['season'] = X['season'].apply(int)
X['game_id'] = X['game_id'].apply(int)
X['mp_date'] = X['mp_date'].apply(int)

In [216]:

X['season'].value_counts()

20172018    1218
20142015    1180
20152016    1180
20112012    1177
20162017    1172
20132014    1166
20092010    1108
20082009    1093
20102011    1091
Name: season, dtype: int64

In [217]:
20180226%10000

226

In [218]:
filt_no_early  = (X['mp_date'].apply(lambda x : x% 10**4) < 900) | (1100 < X['mp_date'].apply(lambda x : x% 10**4))

X = X.loc[filt_no_early, : ].copy()  ## keep games < 900 and > 1100
X.shape

(9104, 27)

,Unnamed: 0,game_id,mp_date,season,home_team,away_team,home_odds,away_odds,home_goals,away_goals,...,FSh%,FSv%,GDiff,GF%,PDO,PENDiff,SF%,SDiff,Sh%,Sv
5857,7467,2014020252,20141115,20142015,MTL,PHI,-160.0,140.0,6.0,3.0,...,-1.456462,0.568475,-0.30,-2.80,-1.03,-20.0,-0.23,-0.01,-1.785336,0.76
5858,7468,2014020253,20141115,20142015,TBL,NYI,-155.0,135.0,5.0,2.0,...,2.153596,0.225057,0.64,5.56,2.99,-3.0,-1.68,-2.02,2.349725,0.64
5859,7469,2014020254,20141115,20142015,NJD,COL,-110.0,-110.0,2.0,3.0,...,0.956409,-0.868548,0.43,4.32,0.64,-3.0,2.43,3.21,1.373739,-0.74
5860,7470,2014020255,20141115,20142015,PIT,NYR,-175.0,155.0,2.0,2.0,...,1.750878,1.476745,1.56,13.46,4.76,9.0,1.13,1.42,2.503294,2.26
5861,7471,2014020256,20141115,20142015,CBJ,SJS,135.0,-155.0,2.0,1.0,...,-0.011364,-2.263427,-1.11,-9.00,-3.05,-2.0,-1.36,-1.66,-0.076481,-2.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9735,11768,2017020596,20171231,20172018,DAL,SJS,-137.0,117.0,6.0,0.0,...,0.749470,-1.089864,-0.26,-2.59,-0.80,-25.0,-0.34,-0.46,0.706558,-1.50
9736,11769,2017020597,20171231,20172018,COL,NYI,-107.0,-113.0,6.0,1.0,...,-0.497050,0.501972,-0.08,-0.62,-0.04,1.0,-0.59,-0.65,-0.583425,0.54
9737,11770,2017020598,20171231,20172018,ANA,ARI,-210.0,180.0,5.0,2.0,...,0.930032,1.695756,1.03,8.95,4.09,-54.0,-2.24,-2.87,1.308561,2.77
9738,11771,2017020599,20171231,20172018,CGY,CHI,-133.0,113.0,4.0,3.0,...,-0.641399,-0.284482,-0.37,-3.33,-1.04,-28.0,-0.33,-0.52,-0.276535,-0.76


In [219]:
y_cols = ['Unnamed: 0', 'game_id', 'mp_date', 'season', 'home_team', 'away_team',
       'home_odds', 'away_odds', 'home_goals', 'away_goals', 'goal_diff_target', 'home_win',
       'settled_in', ]

x_cols = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']
columns_to_scale = ['CF%', 'CSh%', 'CSv%', 'FF%', 'FSh%', 'FSv%', 'GDiff',
       'GF%', 'PDO', 'PENDiff', 'SF%', 'SDiff', 'Sh%', 'Sv']  ##same

In [220]:


x = np.array(X.loc[(X['season'] <= 20152016), x_cols].copy())
  #features test-train
Y = X.loc[(X['season'] <= 20152016), y_cols].copy()
   #targets
y = np.array(Y['home_win']).reshape(-1,1)


              
x_16 = X.loc[(X['season'] == 20162017), x_cols].copy()  #features test-train
Y_16 = X.loc[(X['season'] == 20162017), y_cols].copy()   #targets
y_16 = np.array(Y_16['home_win']).reshape(-1,1)

x_17 = X.loc[(X['season'] > 20162017), x_cols].copy()  #features test-train
Y_17 = X.loc[(X['season'] > 20162017), y_cols].copy()   #targets
y_17 = np.array(Y_17['home_win']).reshape(-1,1)




In [31]:
#checking for leakage?? 20172018 in the training data?

In [32]:
X.loc[(X['season'] >= 20162017), :]['season'].value_counts()

20172018    407
20162017    403
Name: season, dtype: int64

In [33]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts()

20132014    408
20112012    395
20142015    388
20152016    386
20092010    381
20102011    369
20082009    358
Name: season, dtype: int64

In [34]:
X.loc[(X['season'] <= 20152016), :]['season'].value_counts().sum()

2685

In [35]:
print(X.shape)
print(x.shape, Y.shape, y.shape)
print(x_16.shape, y_16.shape)
print(x_17.shape, y_17.shape)
print('sum ', y.shape[0], y_17.shape[0], ' is: ', y.shape[0] + y_17.shape[0])

(3495, 26)
(2685, 14) (2685, 12) (2685, 1)
(403, 14) (403, 1)
(407, 14) (407, 1)
sum  2685 407  is:  3092


In [36]:
Y['season'].value_counts()

20132014    408
20112012    395
20142015    388
20152016    386
20092010    381
20102011    369
20082009    358
Name: season, dtype: int64

In [37]:
Y_16['season'].value_counts()

20162017    403
Name: season, dtype: int64

In [38]:
Y_17['season'].value_counts()

20172018    407
Name: season, dtype: int64

In [39]:
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler, MinMaxScaler 
std_scal = StandardScaler()
mm_scal = MinMaxScaler()

In [51]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


In [52]:
x_test_sc

array([[-0.7235643 ,  1.65934272,  1.90846329, ...,  0.02459198,
         1.34759317,  1.61637573],
       [ 0.49837896, -2.40556644,  0.58823629, ...,  0.25174124,
        -2.23200983,  0.59322356],
       [-0.53437454, -0.56969858,  0.41677738, ..., -0.58681814,
        -0.29213671,  0.09476481],
       ...,
       [ 0.1222252 , -0.87257116, -1.65319514, ..., -0.37670507,
        -0.5712479 , -1.52129093],
       [ 0.57850638, -0.67386903, -0.34579725, ...,  0.75904128,
        -0.43143098, -0.84443641],
       [ 2.81539704,  2.68857895, -0.3907116 , ...,  2.58191417,
         2.81847156, -0.37221233]])

In [221]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.linear_model import LogisticRegression
import xgboost as xgbq
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


clf_A = LogisticRegression(random_state = 2, max_iter = 1000)
clf_B = SVC(random_state = 43, kernel = 'rbf')
clf_C = xgb.XGBClassifier(seed = 44)
clf_D = SVC(probability=True)

for model in [clf_A, clf_B, clf_C, clf_D]: # [lrc,  gnb, lgr, svc,  rfc, bc, gbc, xgbc]:
   
    model.fit(x_train_sc, y_train.ravel())
    y_pred = model.predict(x_test_sc)
  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    
    print(str(model)[0:10], ' TEST error ', acc, 'f1', f1)
    
    y_pred_train_err =  model.predict(x_train_sc) #! careful with this code
    f1_train_err = f1_score(y_train,y_pred_train_err)
    acc_train_err = accuracy_score(y_train, y_pred_train_err)
    print(' training error ', str(model)[0:10], acc_train_err, 'f1', f1) #f1_train_err)
   

LogisticRe  TEST error  0.5679702048417132 f1 0.6741573033707865
 training error  LogisticRe 0.5921787709497207 f1 0.6741573033707865
SVC(random  TEST error  0.5642458100558659 f1 0.6794520547945204
 training error  SVC(random 0.6270949720670391 f1 0.6794520547945204
[11:07:12] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassif  TEST error  0.5363128491620112 f1 0.6078740157480315


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


 training error  XGBClassif 0.9981378026070763 f1 0.6078740157480315
SVC(probab  TEST error  0.5642458100558659 f1 0.6794520547945204
 training error  SVC(probab 0.6270949720670391 f1 0.6794520547945204


In [43]:
#Now check on X_18 
#Now check on X_18 
x_17_sc = std_scal.transform(x_17)
x_test2 = x_17_sc.copy()
y_test2 = y_17


for model in [clf_A, clf_B, clf_C, clf_D]: # [lrc,  gnb, lgr, svc,  rfc, bc, gbc, xgbc]:
   
    #model.fit(x_train_sc, y_train.ravel())
    y_pred2 = model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred2)
    f1 = f1_score(y_test2,y_pred2)
    
    print(model, ' TEST error on 20172018 ', acc, 'f1', f1)
    

LogisticRegression(max_iter=1000, random_state=42)  TEST error on 20172018  0.6216216216216216 f1 0.738095238095238
SVC(random_state=43)  TEST error on 20172018  0.6044226044226044 f1 0.7330016583747927
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=44,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=44,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)  TEST error on 20172018  0.5700245700245701 f1 0.6641074856046065
SVC(probability=True)  TEST error on 20172018  0.6044226044226044 f1 0.7330016583747927


In [44]:
x#Now check on X_18 
x_16_sc = std_scal.transform(x_16)
x_test3 = x_16_sc.copy()
y_test3 = y_16.copy()


for model in [clf_A, clf_B, clf_C, clf_D]: # [lrc,  gnb, lgr, svc,  rfc, bc, gbc, xgbc]:
   
    #model.fit(x_train_sc, y_train.ravel())
    y_pred3 = model.predict(x_test3)
  
    acc = accuracy_score(y_test3, y_pred3)
    f1 = f1_score(y_test3,y_pred3)
    
    print(model, ' TEST error on 20162017 ', acc, 'f1', f1)
    

LogisticRegression(max_iter=1000, random_state=42)  TEST error on 20162017  0.5930521091811415 f1 0.7028985507246377
SVC(random_state=43)  TEST error on 20162017  0.5831265508684863 f1 0.6967509025270757
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=44,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=44,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)  TEST error on 20162017  0.5260545905707196 f1 0.6109979633401221
SVC(probability=True)  TEST error on 20162017  0.5831265508684863 f1 0.6967509025270757


In [45]:
y_17.shape

(407, 1)

In [18]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, f1_score


from sklearn.linear_model import LogisticRegression
import xgboost as xgbq
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 43, kernel = 'rbf')
clf_C = xgb.XGBClassifier(seed = 44)
clf_D = SVC(probability=True)

for model in [clf_A, clf_B, clf_C, clf_D]: # [lrc,  gnb, lgr, svc,  rfc, bc, gbc, xgbc]:
   
    model.fit(x_train_sc, y_train.ravel())
    y_pred = model.predict(x_test_sc)
  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    
    print(str(model)[0:15], ' TEST error ', acc, 'f1', f1)
    
    y_pred_train_err =  model.predict(x_train_sc) #! careful with this code
    f1_train_err = f1_score(y_train,y_pred_train_err)
    acc_train_err = accuracy_score(y_train, y_pred_train_err)
    print(' training error ', model, acc_train_err, 'f1', f1) #f1_train_err)
   

LogisticRegression(random_state=42)  TEST error  0.5676117775354417 f1 0.6586310804993544
 training error  LogisticRegression(random_state=42) 0.5795717987181236 f1 0.6586310804993544
SVC(random_state=43)  TEST error  0.5654307524536533 f1 0.6632868610054923
 training error  SVC(random_state=43) 0.598254466112096 f1 0.6632868610054923
[15:50:45] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=44,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=44,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)  TEST error  0.5179934569247546 f1 0.5778414517669532
 training error  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=

In [ ]:
Now check on X_18 

"<module 'xgboost' from '/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/__init__.py'>"

In [193]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=314, stratify=y)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)
##classifier models
lrc = RidgeClassifier(alpha =0.2)
gnb = GaussianNB()
lgr = LogisticRegression(random_state = 0, C =10)
svc = SVC(kernel = 'rbf', C =10)

#tree-based classifiers
rfc =  RandomForestClassifier(max_depth=10, random_state=0, n_estimators = 40)
bc = BaggingClassifier(n_estimators  = 60, max_samples = 0.85)
gbc = GradientBoostingClassifier(learning_rate =0.1, n_estimators = 40, max_depth =4 )
xgbc = XGBClassifier(n_estimators= 45, eta=0.05)

##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    model.fit(x_train_sc, y_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    y_predt= model.predict(x_train_sc)  
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    acct = accuracy_score(y_train, y_predt)
    f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'TEST: ', acc, f1, 'training : ', acct, f1t)

/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(C=10) TEST:  0.5623836126629422 0.6685472496473908 training :  0.6675977653631285 0.7473460721868366
RidgeClassif TEST:  0.5642458100558659 0.6713483146067416 training :  0.5889199255121043 0.6867683575736077
RandomForest TEST:  0.5363128491620112 0.6289120715350224 training :  0.9120111731843575 0.9263157894736842
GaussianNB() TEST:  0.5381750465549349 0.5947712418300652 training :  0.5819366852886406 0.6310599835661462
LogisticRegr TEST:  0.5716945996275605 0.6769662921348314 training :  0.5889199255121043 0.6858769121309143


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
<ipython-input-193-f7cf8b62249a>:24: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train_sc, y_train)
/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

BaggingClass TEST:  0.5605214152700186 0.6277602523659306 training :  0.9990689013035382 0.9991714995857498


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoos TEST:  0.5661080074487895 0.6666666666666666 training :  0.7337057728119181 0.792
[20:47:09] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifie TEST:  0.5716945996275605 0.6676300578034682 training :  0.7863128491620112 0.8315596330275229


/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/joejohns/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [76]:
x_17_sc = std_scal.transform(x_17)
x_test2 = x_17_sc.copy()
y_test2 = y_17.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'TEST: ', acc, f1)

SVC(C=10) TEST:  0.5823095823095823 0.7079037800687284
RidgeClassif TEST:  0.6167076167076168 0.7346938775510204
RandomForest TEST:  0.5700245700245701 0.6902654867256637
GaussianNB() TEST:  0.547911547911548 0.6377952755905512
LogisticRegr TEST:  0.6167076167076168 0.7346938775510204
BaggingClass TEST:  0.5700245700245701 0.6601941747572815
GradientBoos TEST:  0.597051597051597 0.7191780821917808
XGBClassifie TEST:  0.5945945945945946 0.7140381282495667


In [88]:
x_16_sc = std_scal.transform(x_16)
x_test2 = x_16_sc.copy()
y_test2 = y_16.copy()


##quick checks 
for model in [svc, lrc, rfc, gnb, lgr, bc, gbc, xgbc]:
    #model.fit(x_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test2)
  
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
    acc = accuracy_score(y_test2, y_pred)
    f1 = f1_score(y_test2,y_pred)
   
    #acct = accuracy_score(y_train, y_predt)
    #f1t = f1_score(y_train,y_predt)
  
  
    print(str(model)[0:12], 'TEST: ', acc, f1, 'training : ', acct, f1t)

SVC(C=10) TEST:  0.5707196029776674 0.6778398510242085 training :  0.595903165735568 0.7353658536585366
RidgeClassif TEST:  0.5831265508684863 0.6956521739130435 training :  0.595903165735568 0.7353658536585366
RandomForest TEST:  0.5607940446650124 0.6589595375722543 training :  0.595903165735568 0.7353658536585366
GaussianNB() TEST:  0.5732009925558312 0.6371308016877637 training :  0.595903165735568 0.7353658536585366
LogisticRegr TEST:  0.5905707196029777 0.6994535519125683 training :  0.595903165735568 0.7353658536585366
BaggingClass TEST:  0.5359801488833746 0.6340508806262231 training :  0.595903165735568 0.7353658536585366
GradientBoos TEST:  0.575682382133995 0.6815642458100559 training :  0.595903165735568 0.7353658536585366
XGBClassifie TEST:  0.5558312655086849 0.6629001883239172 training :  0.595903165735568 0.7353658536585366


In [186]:
##regression models now ...

y = Y['goal_diff_target'].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#do standard/minmax scaling on X_train numeric columns ... better to do pipeline? 
x_train_sc = std_scal.fit_transform(x_train)
    
#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


lr = Ridge(alpha=50000) 
rfr = RandomForestRegressor(max_depth=4, random_state=0)
xgbr = XGBRegressor()


##quick checks 
for model in [lr, rfr, xgbr]:
    model.fit(x_train_sc, y_train)
      #model.fit(x_train_sc, y_train.ravel())
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
    y_predt= model.predict(x_train_sc)  
    y_predwt = v_win(y_predt) 
    y_trainw = v_win(y_train)
    y_testw = v_win(y_test)  #same as usual win/loss
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    acct = accuracy_score(y_trainw, y_predwt)
    f1t = f1_score(y_trainw,y_predwt)
    
  
    print(str(model)[0:20], 'TEST: ', acc, f1 ,'training : ', acct, f1t)
    

[0.29148451 0.40520387 0.50665511 0.42079937 0.29841737]
Ridge(alpha=50000) TEST:  0.6424581005586593 0.7818181818181819 training :  0.6303538175046555 0.7732724157624215
[-0.23251353  0.71805064  1.09505305  0.52707299 -0.11915728]
RandomForestRegresso TEST:  0.6182495344506518 0.7381864623243934 training :  0.6662011173184358 0.7654563297350344
[-0.32802024  1.5485265   0.47738376  1.0301529   0.689174  ]
XGBRegressor(base_sc TEST:  0.5698324022346368 0.6627737226277373 training :  0.9436685288640596 0.9533359043578866


In [80]:
def make_win(x):
    if x>=0:
        return 1
    else:
        return 0
    
v_win = np.vectorize(make_win)

In [188]:
##regression models now ...
x_test = x_16.copy()
y_test = Y_16['goal_diff_target'].copy()

#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


##quick checks 
for model in [lr, rfr, xgbr]:
   
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
   
    y_testw = v_win(y_test)  #same as usual win/loss
    
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    
  
    print(str(model)[0:12], 'TEST: ', acc, f1 )


[0.55317694 0.40599628 0.41538467 0.36832525 0.30777493]
Ridge(alpha= TEST:  0.6203473945409429 0.7656967840735069
[ 0.46260171  0.75274141  0.80154869  0.28970433 -0.58376368]
RandomForest TEST:  0.6228287841191067 0.7379310344827585
[-0.20716095 -2.5455654   0.30311152  1.9340824  -0.70554775]
XGBRegressor TEST:  0.5707196029776674 0.6560636182902585


In [187]:
##regression models now ...
x_test = x_17.copy()
y_test = Y_17['goal_diff_target'].copy()

#fit the scaler from train portion to the test portion 
x_test_sc = std_scal.transform(x_test)


##quick checks 
for model in [lr, rfr, xgbr]:
   
    y_pred= model.predict(x_test_sc)
    print(y_pred[0:5])
    y_predw = v_win(y_pred)
   
    y_testw = v_win(y_test)  #same as usual win/loss
    
    acc = accuracy_score(y_testw, y_predw)
    f1 = f1_score(y_testw,y_predw)
    
  
    print(str(model)[0:12], 'TEST: ', acc, f1 )

[0.31938552 0.29756373 0.42518078 0.46607625 0.27714656]
Ridge(alpha= TEST:  0.6167076167076168 0.7629179331306991
[ 0.26194843  0.17763043  0.4769733   0.75905672 -0.70724261]
RandomForest TEST:  0.601965601965602 0.7335526315789472
[ 0.75880516  1.538813    1.0374336   2.035714   -0.8959621 ]
XGBRegressor TEST:  0.5552825552825553 0.6552380952380952
